# Part IV: Survival analysis

## 1. Load the required packages

## 2. Load the saved MOFA and extract the quantities of interest

## 3. Create Hazard Ratio Plot

## 4. Create Kaplan-Meier-Plot and plot survival over time (factor specific)

In [ ]:
# Make dataframe for hazard ratio plot
df <- data.frame(
  factor = factor(rownames(coef), levels = rev(rownames(coef))),
  p      = coef[,"Pr(>|z|)"], 
  coef   = coef[,"exp(coef)"], 
  lower  = s[["conf.int"]][,"lower .95"], 
  higher = s[["conf.int"]][,"upper .95"]
)

# Plot hazard ratio per factor
ggplot(df, aes(x=factor, y=coef, ymin=lower, ymax=higher)) +
  geom_pointrange( col='#619CFF') + 
  coord_flip() +
  scale_x_discrete() + 
  labs(y="Hazard Ratio", x="") + 
  geom_hline(aes(yintercept=1), linetype="dotted") +
  theme_bw()

In [ ]:
serialized_mofa_file = file.path(getwd(), "../MOFAobject.LUAD.hdf5")
MOFAobject.trained <- load_model(serialized_mofa_file)

SurvObject <- Surv(MOFAobject.trained@samples_metadata$days_to_death, MOFAobject.trained@samples_metadata$vital_status)
Z <- get_factors(MOFAobject.trained)[[1]]

fit <- coxph(SurvObject ~ Z) 
fit

s <- summary(fit)
coef <- s[["coefficients"]]

In [ ]:
library(MOFA2)
library(survival)
library(survminer)

In [ ]:
df <- data.frame(
  time = SurvObject[,1], 
  event = SurvObject[,2], Z1 = Z[,1]
)
cut <- surv_cutpoint(df, variables='Z1')
df$FactorCluster <- df$Z1 > cut$cutpoint$cutpoint
fit <- survfit(Surv(time, event) ~ FactorCluster, df)

ggsurvplot(fit, data = df,
  conf.int = TRUE, pval = TRUE,
  fun = function(y) y * 100,
  legend = "top", legend.labs = c(paste("low LF 1"), paste("high LF 1")),
  xlab = "Overall Survival", ylab="Survival probability (%)", title= "Factor 1"
)$plot